<a href="https://colab.research.google.com/github/sean-halpin/ml_tweepy_proj/blob/main/twitter_sentiment_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [2]:
BEARER_TOKEN = ""

In [13]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [3]:
import tweepy

client = tweepy.Client(
    bearer_token=BT
)

In [11]:
search_result = client.search_recent_tweets(query="russia -is:retweet",max_results=10)

In [32]:
import pandas as pd


In [33]:
list_of_lists = []
i=0
for sr in search_result.data:

  print("{}:{}".format(i,sr.text))
  i+=1
  text = sr.text
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      print(f"\t{i+1}) {l} {np.round(float(s), 4)}")
  print()
  list_of_lists.append([sr.text, str(labels[ranking[0]])])

0:@hardcorehunte17 @phildstewart Russia didn’t give anyone much of a choice.  Sucks to be RU.
	1) negative 0.8852
	2) neutral 0.1051
	3) positive 0.0096

2:L’ambaixada de #Rússia recolzant públicament i explícitament (amb citació personal inclosa) la postura de C. Montaner.

No puc imaginar una vergonya més gran.

L’ambaixada russa avisa Andorra que les relacions bilaterals queden malmeses via @AltaveuAndorra https://t.co/lEUlPJikOB
	1) neutral 0.8239
	2) negative 0.1304
	3) positive 0.0456

2:Even without Russia's small drums, the tech investment beat goes on  The tech market has plenty of problems to contend with in 2022 and beyond but demand is not currently one of them. What is complicating prognostications for the wider global ICT market... https://t.co/pLG6mU2QUX
	1) negative 0.5969
	2) neutral 0.3574
	3) positive 0.0457

2:12th day of special military operation by Russian Federation inside T-Borderland (#Ukraine). Look back last 3 days. #Donbass #Russia 
https://t.co/qxdF2e4geV


In [37]:
cols = ["tweet", "sentiment"]
df = pd.DataFrame(list_of_lists, columns=cols)
df.head()

,tweet,sentiment
0,@hardcorehunte17 @phildstewart Russia didn’t g...,negative
1,L’ambaixada de #Rússia recolzant públicament i...,neutral
2,"Even without Russia's small drums, the tech in...",negative
3,12th day of special military operation by Russ...,neutral
4,Latvian Foreign Minister Edgars Rinkevics on M...,neutral


In [38]:
df.to_csv("tweets_annotated.csv", header=cols, index=False)